In [ ]:
# Notebook to display data for selected highway links flow, V/C, and speeds
# each as a static map layer overlayed on top of a map of Massachusetts towns
# using the matplotlib library.
# 
# Tabular data was previously calculated by sample_highway_links_report.ipypnb, and saved in CSV format.
#
import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

In [ ]:
# Directory in which user's output CSV report data was saved - it will now be our *input* directory
my_sandbox_dir = r'S:/my_modx_output_dir/'

In [ ]:
# Name of CSV file with volume, V/C, and speed data for selected links - it will now be our *input* CSV file
csv_fn = 'links_report_base_scenario.csv'

In [ ]:
# Fully-qualified pathname to CSV file
fq_csv_fn = my_sandbox_dir + csv_fn

In [ ]:
links_data_df = pd.read_csv(fq_csv_fn, delimiter=',')

In [ ]:
links_data_df

In [ ]:
# List of the IDs for the model network links for which data is reported in the input CSV file
links_list = links_data_df['ID1'].to_list()

In [ ]:
# Directory in which the spatial data for the model network links is stored (both shapefile and GeoJSON formats)
links_spatial_data_dir = r'G:/Data_Resources/modx/statewide_links_shapefile/'

In [ ]:
# Load the links shapefile into a geopandas dataframe 
# NOTE: This version of the shapefile is in EPSG4326, i.e., WGS84
links_shapefile_fn = 'Statewide_Links_2018_BK_EPSG4326.shp'
fq_links_shapefile_fn = links_spatial_data_dir + links_shapefile_fn
links_gdf = gp.read_file(fq_links_shapefile_fn)
links_gdf.set_index("ID")

In [ ]:
# Filter the links geodataframe to only the links of interest
filtered_links_gdf = links_gdf[links_gdf['ID'].isin(links_list)] 

In [ ]:
filtered_links_gdf

In [ ]:
# Join the geo-data frame for the links with the "links_data_df", which contains the computed data about these links
join_df = filtered_links_gdf.join(links_data_df.set_index("ID1"), on="ID")

In [ ]:
join_df

In [ ]:
# Return the bounding box of all the features in a geo-dataframe.
# The bounding box is returned as a dictionary with the following keys: { 'minx', 'miny', 'maxx', 'maxy'}.
#
def bbox_of_gdf(gdf):
    bounds_tuples = gdf['geometry'].map(lambda x: x.bounds)
    bounds_dicts = []
    for t in bounds_tuples:
        temp = { 'minx' : t[0], 'miny' : t[1], 'maxx' : t[2], 'maxy' : t[3] }
        bounds_dicts.append(temp)
    # end_for
    bounds_df = pd.DataFrame(bounds_dicts)
    minx = bounds_df['minx'].min()
    miny = bounds_df['miny'].min()
    maxx = bounds_df['maxx'].max()
    maxy = bounds_df['maxy'].max()
    retval = { 'minx' : minx, 'miny' : miny, 'maxx' : maxx, 'maxy' : maxy }
    return retval
# end_def bbox_of_gdf()

# Given a dictonary of the form  'minx', 'miny', 'maxx', 'maxy'} representing a geographic bounding box,
# return the center point as a dictionary with the keys { 'x' , 'y' }.
def center_of_bbox(bbox):
    center_x = bbox['minx'] + (bbox['maxx'] - bbox['minx']) / 2
    center_y = bbox['miny'] + (bbox['maxy'] - bbox['miny']) / 2
    retval = { 'x' : center_x, 'y' : center_y }
    return retval
# end_def center_of_bbox()

In [ ]:
# Get the bounding box of the selected links
bbox = bbox_of_gdf(join_df)

In [ ]:
# Get the center point of that bbox
center_pt = center_of_bbox(bbox)

In [ ]:
# Plot (i.e., map) extent for matplotlib.
# The extent for matplotlib is defined as [minx, maxx, miny, maxy]
matplotlib_extent = [ bbox['minx'], bbox['maxx'], bbox['miny'], bbox['maxy'] ]

In [ ]:
matplotlib_extent

In [ ]:
# Directory containing miscellaneous reference data
misc_reference_data_dir = r'G:/Data_Resources/modx/misc_reference_data/'

In [ ]:
# Load the MassGIS TOWNS_POLYM shapefile into a geopandas dataframe 
# NOTE: This version of the shapefile is in EPSG4326, i.e., WGS84
towns_shapefile_fn = 'towns_polym_EPSG4326.shp'
fq_towns_shapefile_fn = misc_reference_data_dir + towns_shapefile_fn
towns_gdf = gp.read_file(fq_towns_shapefile_fn)
towns_gdf.set_index("town_id")

In [ ]:
# Make a static map of speed during the AM period overlayed on the towns layer
base = towns_gdf.plot(color='white', edgecolor='black')
join_df.plot("Speed_am", ax=base, figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.xlim((bbox['minx'], bbox['maxx']))
plt.ylim((bbox['miny'], bbox['maxy']))
plt.title('Speed in AM')
plt.show()